In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
import json
import pandas as pd
import re
import os
from copy import copy
import math


from transformers import AutoTokenizer, AutoModelWithLMHead, get_scheduler
import torch
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

In [ ]:
#DATA_DIR = '../data'
DATA_DIR = ''
df = pd.read_csv(os.path.join(DATA_DIR, 'kremlin_ner_dataset.csv'))

In [ ]:
for column in ['title_ner', 'text_ner']:
    df[column] = df[column].apply(json.loads)

In [ ]:
SELECTED_ENTITIES = ['PER', 'ORG']

BOS_TAG = 'BOS'
EOS_TAG = 'EOS'
TITLE_TAG = 'TITLE'
TEXT_TAG = 'TEXT'

In [ ]:
def join_entities(title_ner, text_ner, selected_entities=SELECTED_ENTITIES):
    '''
    Для выбранных типов сущностей берет значение из сущностей заголовка (при наличии).
    Если сущностей нет в заголовке, берет из текста
    '''
    joined_entities = {}
    for entity_type in selected_entities:
        for entities in [title_ner, text_ner]:
            if entity_type in entities:
                joined_entities[entity_type] = entities[entity_type]
                break
    return joined_entities

In [ ]:
df['selected_entities'] = df.apply(
    lambda row: join_entities(row['title_ner'], row['text_ner']),
    axis=1
)

In [ ]:
df['selected_entities'].apply(len).describe()

count    2380.000000
mean        0.944118
std         0.601095
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: selected_entities, dtype: float64

Сущности не выделены меньше, чем для четверти примеров

In [ ]:
def prepare_text_for_gpt(text:str):
    '''
    Функция добавляет пробелы после разделителей строки и точек, если пробелов там не было,
    потому что при токенизации пробел является частью первого токена слова.
    Эмбединги для случаев "заглавная буква идет не после пробела" предобучены хуже.
    '''
    text = re.sub('\n', ' ', text)
    text = re.sub(re.escape('.'), '. ', text)
    # если были добавлены двойные пробелы, убираем их
    text = re.sub(' +', ' ', text)
    return text

def format_prompt(row:pd.core.series.Series):
    prompt = ''
    for entity_tag in SELECTED_ENTITIES:
        entity_content = ', '.join(row['selected_entities'].get(entity_tag, []))
        prompt += f'{entity_tag} {entity_content} '
    prompt = f"{BOS_TAG} {prompt}{TITLE_TAG} {row['title']} {TEXT_TAG} {row['text']} {EOS_TAG}"
    return prepare_text_for_gpt(prompt)

In [ ]:
df['prompt'] = df.apply(format_prompt, axis=1)

Посмотрим распределение длин токенизированных текстов

In [ ]:
PRETRAINED_MODEL_NAME = 'sberbank-ai/rugpt3small_based_on_gpt2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.add_special_tokens({'bos_token':BOS_TAG,
                              'eos_token':EOS_TAG,
                              'pad_token': '[PAD]',
                              'additional_special_tokens':[TITLE_TAG, TEXT_TAG, *SELECTED_ENTITIES]})

7

In [ ]:
tokenized = []
prompts = list(df['prompt'])
encoded_prompts_lens = []
tokenization_batch_size = 100

for batch_idx in range(math.ceil(len(prompts)/tokenization_batch_size)):
    batch_start = batch_idx * tokenization_batch_size
    batch_end = (batch_idx+1) * tokenization_batch_size
    batch = prompts[batch_start:batch_end]
    # при большом объеме данных стоило бы сохранить результат токенизации и один раз допадить, 
    # но сейчас токенизация не занимает много времени
    encoded_prompts_lens += [len(i) for i in tokenizer(batch)['input_ids']]
df['encoded_prompts_lens'] = encoded_prompts_lens

In [ ]:
df['encoded_prompts_lens'].describe()

count    2380.000000
mean      192.957563
std        60.398859
min        64.000000
25%       144.000000
50%       192.000000
75%       231.000000
max       534.000000
Name: encoded_prompts_lens, dtype: float64

Тексты не превышают ограничения языковой модели, но я все равно уберу самые длинные, чтобы избежать превышения памяти cuda

In [ ]:
length_treshold = 250
df = df[df['encoded_prompts_lens']<=length_treshold]

In [ ]:
df['encoded_prompts_lens'].describe()

count    1993.000000
mean      174.416959
std        44.061091
min        64.000000
25%       135.000000
50%       179.000000
75%       211.000000
max       250.000000
Name: encoded_prompts_lens, dtype: float64

In [ ]:
train, test = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
# для ускорения обучения полезно сделать так, чтобы в батче подавались тексты примерно одной длины (будет меньше падингов)
# ascending=False, чтобы в случае превышения памяти на самых длинных текстах сразу увидеть эту проблему
# стоило бы сделать так, чтобы короткие и длинные батчи подавались в перемешку, но (зачеркнуто: мне лень) 
# при небольшом количестве итераций в эпохе вряд ли модель успеет переобучиться на длину последних (коротких) текстов
train.sort_values('encoded_prompts_lens', ascending=False, inplace=True)

In [ ]:
train.soft_values('encoded_prompts_lens')

In [ ]:
train_batch_size = 5
train_dataloader = torch.utils.data.DataLoader(list(train['prompt']), batch_size=train_batch_size)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
num_epochs = 25
num_training_steps = num_epochs*len(train_dataloader)

In [ ]:
gpt = AutoModelWithLMHead.from_pretrained(PRETRAINED_MODEL_NAME).to(device).train()
gpt.resize_token_embeddings(len(tokenizer))
optimizer = torch.optim.AdamW(gpt.parameters(), lr=5e-5)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,



В генеративных задачах модель с лучшими метриками не обязательно выдает лучший с точки зрения соответствия целевой задаче результат.
Для ускорения работы сначала дообучу модель, а потом уже посчитаю метрики на тесте без подсчета метрик для чекпойнтов.

In [ ]:
from google.colab import drive
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [ ]:
SAVE_PATH = 'gdrive/MyDrive/gpt_small_president_letter'

In [ ]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        tokenized_batch = tokenizer(list(batch), return_tensors='pt', padding=True).to(device)
        outputs = gpt(**tokenized_batch, labels=tokenized_batch['input_ids'])
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    gpt.save_pretrained(SAVE_PATH)

  0%|          | 0/8975 [00:00<?, ?it/s]

Проверка на адекватность результата (более детальный анализ и подбор параметров будет в другой тетрадке)

In [ ]:
gpt.eval()

In [ ]:
test.reset_index(inplace=True)

In [ ]:
real_prompt = test['prompt'].loc[0]

In [ ]:
real_prompt

'BOS PER Сергея Крамаренко ORG Вооружённых Сил TITLE Родным и близким Героя Советского Союза Сергея Крамаренко TEXT С глубоким прискорбием узнал о кончине Сергея Макаровича Крамаренко. Сергей Макарович Крамаренко был мужественным, сильным духом человеком, фронтовиком, участником важнейших сражений Великой Отечественной. А после войны он посвятил себя развитию Вооружённых Сил страны, патриотическому воспитанию молодёжи, укреплению замечательных традиций ветеранского движения. Светлая память о Сергее Крамаренко, Герое Советского Союза, легендарном лётчике, навсегда сохранится в сердцах его родных, сослуживцев, коллег и друзей. EOS'

In [ ]:
prompt_meta = real_prompt.split(TITLE_TAG)[0] + TITLE_TAG
prompt_meta_with_beginning = real_prompt.split(TEXT_TAG)[0] + TEXT_TAG

In [ ]:
prompt_meta

'BOS PER Сергея Крамаренко ORG Вооружённых Сил TITLE'

In [ ]:
prompt_meta_with_beginning

'BOS PER Сергея Крамаренко ORG Вооружённых Сил TITLE Родным и близким Героя Советского Союза Сергея Крамаренко TEXT'

In [ ]:
def generate(prompt):
   encoding = tokenizer([prompt], return_tensors='pt').to(device)
   output = gpt.generate(**encoding, 
                         min_length=50,
                         max_length=250,
                         bad_words_ids=[[tokenizer.pad_token_id]],
                         eos_token_id=tokenizer.eos_token_id)
   return tokenizer.decode(output[0])

In [ ]:
generate(prompt_meta)

Setting `pad_token_id` to `eos_token_id`:50259 for open-end generation.


'BOS PER Сергея Крамаренко ORG Вооружённых Сил TITLE Родным и близким Сергея Крамаренко TEXT Примите глубокие соболезнования в связи с кончиной Сергея Леонидовича Крамаренко. Сергей Леонидович был опытным руководителем, честным, порядочным, очень отзывчивым и доброжелательным человеком. Все, кто знал Сергея Леонидовича, ценили его за высокую компетентность, силу воли и мужество, честность и порядочность, верность своим принципам и долгу. Светлая память о Сергее Леонидовиче Крамаренко навсегда сохранится в сердцах близких, коллег, друзей. EOS'

In [ ]:
generate(prompt_meta_with_beginning)

Setting `pad_token_id` to `eos_token_id`:50259 for open-end generation.


'BOS PER Сергея Крамаренко ORG Вооружённых Сил TITLE Родным и близким Героя Советского Союза Сергея Крамаренко TEXT Примите глубокие соболезнования в связи с кончиной Сергея Леонидовича Крамаренко. Сергей Леонидович был выдающимся представителем прославленного поколения победителей, волевым, энергичным человеком, настоящим патриотом и патриотом. Он всегда стремился приносить пользу Родине, достойно решать ответственные задачи, добивался успеха в сложнейшей оперативной работе. За годы ответственной работы, в том числе в переломное для нашей страны время Сергей Леонидович внёс значимый личный вклад в укрепление Вооружённых Сил, укрепление обороноспособности и национальной безопасности страны. Светлая память о Сергее Леонидовиче Крамаренко навсегда сохранится в наших сердцах. EOS'

Результат настолько хороший, что это кажется подозрительным. Модель могла знать отчество и титул Крамаренко, но вряд ли могла угадать уже при первой генерации (без названия), что президент обращается к близким Крамаренко в связи с его смертью. 

Возможно, в датасете были дубли с разными ссылками. Попробую проверить на данных, которых точно не было в трейне

In [64]:
dummy_prompt = 'BOS PER Сергей Иванов ORG Психиатрическая больница №1 TITLE'

In [65]:
generate(dummy_prompt)

Setting `pad_token_id` to `eos_token_id`:50259 for open-end generation.


'BOS PER Сергей Иванов ORG Психиатрическая больница №1 TITLE Сотрудникам и ветеранам Психиатрической больницы №1 TEXT Уважаемые друзья! Поздравляю вас с большой, знаменательной датой – 100-летием Психиатрической больницы №1. Одна из крупнейших в стране, знаменитая Психиатрическая больница имени С. С. Ланового, по праву славится богатой историей, замечательными традициями, гостеприимным, радушным приёмом и радушием, с которым здесь встречают пациентов. Здесь проводится большая, многогранная работа, направленная на профилактику и оздоровление людей с ограниченными возможностями по здоровью, совершенствование профильного законодательства. И конечно, отмечу ваше активное участие в реализации востребованных благотворительных, просветительских, патриотических проектов. Уверен, что коллектив больницы и впредь будет беречь и развивать замечательные традиции своих предшественников, достойно решать стоящие перед ней задачи. Желаю вам успехов и всего наилучшего. EOS'

Кажется, что модель видела слишком много обращений к военным ("сотрудникам и ветеранам").
Название организации повторятся правильно, имя человека не использовано, но нет и неверного имени (имени С. С. Ланового - не имя человека, которого поздравляют).

Интересно, что психиатр С.Лановая действительно существует)